In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e7/sample_submission.csv
/kaggle/input/playground-series-s4e7/train.csv
/kaggle/input/playground-series-s4e7/test.csv


**IMPORT LIBRARIES**

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

LOAD TRAIN DATA

In [3]:
df = pd.read_csv('/kaggle/input/playground-series-s4e7/train.csv')

LOAD TEST DATA

In [4]:
TD = pd.read_csv('/kaggle/input/playground-series-s4e7/test.csv')

In [5]:
df.shape

(11504798, 12)

In [6]:
df.head(9)

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,0,Male,21,1,35.0,0,1-2 Year,Yes,65101.0,124.0,187,0
1,1,Male,43,1,28.0,0,> 2 Years,Yes,58911.0,26.0,288,1
2,2,Female,25,1,14.0,1,< 1 Year,No,38043.0,152.0,254,0
3,3,Female,35,1,1.0,0,1-2 Year,Yes,2630.0,156.0,76,0
4,4,Female,36,1,15.0,1,1-2 Year,No,31951.0,152.0,294,0
5,5,Female,31,1,47.0,1,< 1 Year,No,28150.0,152.0,197,0
6,6,Male,23,1,45.0,1,< 1 Year,No,27128.0,152.0,190,0
7,7,Female,47,1,8.0,0,1-2 Year,Yes,40659.0,26.0,262,1
8,8,Female,26,1,28.0,1,< 1 Year,No,31639.0,152.0,36,0


In [7]:
df['Region_Code'].unique()

array([35. , 28. , 14. ,  1. , 15. , 47. , 45. ,  8. , 11. ,  3. , 10. ,
       12. , 37. , 50. , 30. , 46. , 29. , 16. , 26. , 25. , 41. , 48. ,
        7. ,  4. , 24. , 36. , 33. ,  2. ,  9. , 27. , 18. , 43. , 49. ,
       17. , 13. , 38. ,  6. , 21. , 23. ,  0. , 19. ,  5. , 39. , 44. ,
       20. , 40. , 22. , 32. , 42. , 31. , 34. , 51. , 52. , 39.2])

In [8]:
df.isnull().sum()

id                      0
Gender                  0
Age                     0
Driving_License         0
Region_Code             0
Previously_Insured      0
Vehicle_Age             0
Vehicle_Damage          0
Annual_Premium          0
Policy_Sales_Channel    0
Vintage                 0
Response                0
dtype: int64

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
le_col = ['Gender','Vehicle_Damage']
le = LabelEncoder()


for col in le_col:
    df[col] = le.fit_transform(df[col])

In [10]:
ohe = OneHotEncoder()
encoded_vehicle_age = ohe.fit_transform(df[['Vehicle_Age']]).toarray()

encoded_vehicle_age_df = pd.DataFrame(encoded_vehicle_age, columns=ohe.get_feature_names_out(['Vehicle_Age']))

df = pd.concat([encoded_vehicle_age_df, df.drop('Vehicle_Age', axis=1)], axis=1)

In [11]:
df

,Vehicle_Age_1-2 Year,Vehicle_Age_< 1 Year,Vehicle_Age_> 2 Years,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1.0,0.0,0.0,0,1,21,1,35.0,0,1,65101.0,124.0,187,0
1,0.0,0.0,1.0,1,1,43,1,28.0,0,1,58911.0,26.0,288,1
2,0.0,1.0,0.0,2,0,25,1,14.0,1,0,38043.0,152.0,254,0
3,1.0,0.0,0.0,3,0,35,1,1.0,0,1,2630.0,156.0,76,0
4,1.0,0.0,0.0,4,0,36,1,15.0,1,0,31951.0,152.0,294,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11504793,1.0,0.0,0.0,11504793,1,48,1,6.0,0,1,27412.0,26.0,218,0
11504794,0.0,1.0,0.0,11504794,0,26,1,36.0,0,1,29509.0,152.0,115,1
11504795,0.0,1.0,0.0,11504795,0,29,1,32.0,1,0,2630.0,152.0,189,0
11504796,1.0,0.0,0.0,11504796,0,51,1,28.0,0,1,48443.0,26.0,274,1


In [12]:
df.drop(columns=['id'])

,Vehicle_Age_1-2 Year,Vehicle_Age_< 1 Year,Vehicle_Age_> 2 Years,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1.0,0.0,0.0,1,21,1,35.0,0,1,65101.0,124.0,187,0
1,0.0,0.0,1.0,1,43,1,28.0,0,1,58911.0,26.0,288,1
2,0.0,1.0,0.0,0,25,1,14.0,1,0,38043.0,152.0,254,0
3,1.0,0.0,0.0,0,35,1,1.0,0,1,2630.0,156.0,76,0
4,1.0,0.0,0.0,0,36,1,15.0,1,0,31951.0,152.0,294,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11504793,1.0,0.0,0.0,1,48,1,6.0,0,1,27412.0,26.0,218,0
11504794,0.0,1.0,0.0,0,26,1,36.0,0,1,29509.0,152.0,115,1
11504795,0.0,1.0,0.0,0,29,1,32.0,1,0,2630.0,152.0,189,0
11504796,1.0,0.0,0.0,0,51,1,28.0,0,1,48443.0,26.0,274,1


In [13]:
columns_to_scale = ['Age', 'Region_Code', 'Annual_Premium', 'Policy_Sales_Channel', 'Vintage']


sc = StandardScaler()
df[columns_to_scale] = sc.fit_transform(df[columns_to_scale])

In [14]:
df

,Vehicle_Age_1-2 Year,Vehicle_Age_< 1 Year,Vehicle_Age_> 2 Years,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1.0,0.0,0.0,0,1,-1.159410,1,0.660528,0,1,2.105145,0.214202,0.288852,0
1,0.0,0.0,1.0,1,1,0.307897,1,0.121718,0,1,1.728962,-1.599414,1.551675,1
2,0.0,1.0,0.0,2,0,-0.892627,1,-0.955902,1,0,0.460756,0.732378,1.126566,0
3,1.0,0.0,0.0,3,0,-0.225669,1,-1.956550,0,1,-1.691389,0.806403,-1.099003,0
4,1.0,0.0,0.0,4,0,-0.158974,1,-0.878929,1,0,0.090529,0.732378,1.626694,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11504793,1.0,0.0,0.0,11504793,1,0.641376,1,-1.571685,0,1,-0.185319,-1.599414,0.676451,0
11504794,0.0,1.0,0.0,11504794,0,-0.825931,1,0.737501,0,1,-0.057878,0.732378,-0.611378,1
11504795,0.0,1.0,0.0,11504795,0,-0.625844,1,0.429609,1,0,-1.691389,0.732378,0.313859,0
11504796,1.0,0.0,0.0,11504796,0,0.841463,1,0.121718,0,1,1.092793,-1.599414,1.376630,1


In [15]:
X = df.drop(columns=['Response','id']).values
y = df['Response'].values

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)


In [17]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

In [18]:
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.83


In [19]:
model = GaussianNB()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

In [20]:
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.72


In [25]:
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

In [26]:
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.88


In [32]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(units=64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [33]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=1024, callbacks=[early_stopping])

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy}")

Epoch 1/10
8989/8989 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - accuracy: 0.8768 - loss: 0.2768 - val_accuracy: 0.8769 - val_loss: 0.2680
Epoch 2/10
8989/8989 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - accuracy: 0.8771 - loss: 0.2658 - val_accuracy: 0.8769 - val_loss: 0.2692
Epoch 3/10
8989/8989 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - accuracy: 0.8769 - loss: 0.2659 - val_accuracy: 0.8769 - val_loss: 0.2686
Epoch 4/10
8989/8989 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - accuracy: 0.8768 - loss: 0.2658 - val_accuracy: 0.8769 - val_loss: 0.2685
71905/71905 ━━━━━━━━━━━━━━━━━━━━ 89s 1ms/step - accuracy: 0.8773 - loss: 0.2679
Test Accuracy: 0.8768861889839172
